# 데이터셋

쇼핑 상품 분류 대회에서는 총 3개의 데이터셋을 제공합니다.

    train
    dev
    test

각 데이터셋은 hdf5 포맷으로 저장되어 있습니다. hdf5 파일을 사용하는 방법에 대해서는 대회 안내 페이지를 참고하세요.

train은 총 8,134,818개의 샘플이 포함되어있고, dev는 507,783개, test는 1,526,523개의 샘플이 포함되어 있습니다. train은 실험하는 목적으로 정답 카테고리값과 함께 제공되나 dev와 test 샘플에는 정답 카테고리 값이 없습니다.

dev 샘플에 대한 예측한 카테고리 정보는 대회 기간동안 예측 파일 제출을 통해 정확도를 리더보드에서 확인할 수 있습니다. test 샘플에 대한 예측 결과는 대회 종료 후에 공개되며 대회 종료전까지 여러번 제출이 가능하나, 마지막 제출한 결과만 사용되며 test 샘플에 대한 예측 결과가 대회 최종 순위를 결정하는데 사용됩니다.

## 데이터셋 설명

모든 샘플은 중카테고리까지의 분류는 있지만 소나 세카테고리까지의 분류는 없을 수 있습니다. 카테고리정보는 숫자로 치환되어 있으며 실제 카테고리명은 cate1.json에 있습니다. b, m, s, d는 각각 대/중/소/세 카테고리를 의미하며, train의 샘플 데이터중에 카테고리 정보가 -1인 것은 해당 카테고리 분류 결과가 없다는 뜻이며, dev나 test은 평가를 위해 카테고리 정보가 모두 -1로 제공됩니다.

카테고리내 ID는 모두 1부터 시작합니다. (-1 제외)

In [1]:
# 필요한 Package import
from catekitten.data import Loader, get_category_map, split_pos
from catekitten.transform import PosTokenizer
from catekitten.evaluate import arena_accuracy_score
from catekitten import tf_glove

from pprint import pprint
from collections import defaultdict
from eunjeon import Mecab

from gensim.models import FastText, Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import h5py

import tensorflow as tf
import keras

from tensorflow.keras import Sequential
from tensorflow.keras import layers

C:\Users\iam\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\iam\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\iam\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
# Dataset object 생성
dataset = Loader("data/prep/textonly.h5")
y_label = get_category_map("data/raw/cate1.json")

In [3]:
# Dataset columns 확인
pprint(dataset.columns)

['bcateid',
 'dcateid',
 'mcateid',
 'price',
 'scateid',
 'brand',
 'maker',
 'model',
 'pid',
 'product',
 'updttm']


In [4]:
# category id 확인
pprint(list(y_label['bcateid'].values())[:5])
pprint(list(y_label['mcateid'].values())[:5])
pprint(list(y_label['scateid'].values())[:5])
pprint(list(y_label['dcateid'].values())[:5])

['가공식품/과자/초콜릿', '수납/정리/선반', '디카/캠코더/주변기기', '침구/커튼/카페트', '데스크탑/모니터/PC부품']
['김치냉장고', '권투용품', '프라이팬/용품', '야구', 'CPU']
['', '탱크 RC', '북엔드', '로비의자', '한복/생활한복']
['', 'MP3 케이블/충전기', 'MP3 케이스/파우치', '전동칫솔', '춘천시']


In [5]:
len(sorted(y_label['bcateid'].keys()))

57

In [6]:
print("\nPrice:")
pprint(dataset["price", :5])
print("\nupdttm:")
pprint(dataset["updttm", :5])
print("Brand:")
pprint(dataset["brand", :5])
print("\nMaker:")
pprint(dataset["maker", :5])
print("\nModel:")
pprint(dataset["model", :5])
print("\nProduct:")
pprint(dataset["product", :5])


Price:
0    16520
1    20370
2       -1
3    16280
4       -1
Name: price, dtype: int32

updttm:
0    1519690229.0
1    1524959419.0
2    1524705794.0
3    1524354312.0
4    1524521783.0
Name: updttm, dtype: object
Brand:
0    퍼즐라이프
1     바보사랑
2     크리비아
3      잭앤질
4         
Name: brand, dtype: object

Maker:
0    상품상세설명 참조
1    MORY|해당없음
2             
3       ㈜크리스패션
4           기타
Name: maker, dtype: object

Model:
0                           퍼즐라이프 직소퍼즐 바다거북의 여행
1    아이폰6S/6S+ tree farm101 - 다이어리케이스|아이폰6S/6S+
2                       크리비아 기모 3부 속바지 GLG4314P
3     [잭앤질] 남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA
4                              SD코드프리혈당시험지[50매]
Name: model, dtype: object

Product:
0                      직소퍼즐 - 1000조각 바다거북의 여행 (PL1275)
1    [모리케이스]아이폰6S/6S+ tree farm101 - 다이어리케이스[바보사랑][...
2                              크리비아 기모 3부 속바지 GLG4314P
3        [하프클럽/잭앤질]남성 솔리드 절개라인 포인트 포켓 팬츠 31133PT002_NA
4                          코드프리혈당시험지50매/코드프리시험지/최장유효기간
Name: product, dtype: obj

In [7]:
text_attrs = ["brand", "maker", "model", "product"]
tagger = PosTokenizer()
x = np.zeros([200000, 400])

for i, text_attr in enumerate(text_attrs):
    print(text_attr)
    attr_text = tagger.transform(dataset[text_attr, :200000]) 
    attr_text = attr_text.map(split_pos)
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(attr_text)]
    model = Doc2Vec(documents, workers=4)
    x[:,i*100:(i+1)*100] = np.vstack(attr_text.map(model.infer_vector))
    model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

brand
maker
model
product


In [11]:
y = dataset['bcateid', :200000]
y = keras.utils.to_categorical(y, num_classes=57)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=39)

In [12]:
with tf.device("/gpu:0"):
    pooled_tensors = []
    model = Sequential()
    model.add(layers.Embedding(1000, 100, input_length=400))
    
    for filter_size in self.filter_sizes:
        x_i = model.add(layers.Conv1D(self.num_filters, filter_size, activation='elu', **self.conv_kwargs)
        x_i = layers.GlobalMaxPooling1D
        pooled_tensors.append(x_i)

    x = pooled_tensors[0] if len(self.filter_sizes) == 1 else concatenate(pooled_tensors, axis=-1)

#     model.add(layers.Dense(512, activation='relu', input_dim=400))
#     model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(512, activation='relu'))
#     model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(512, activation='relu', input_dim=400))
#     model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(512, activation='relu'))
#     model.add(layers.Dropout(0.5))
    model.add(layers.Dense(57, activation='softmax'))
    optimizer = tf.keras.optimizers.Nadam(1e-4)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=[keras.metrics.categorical_accuracy]
    )
    model.summary()
    model.fit(
        x_train,
        y_train,
        validation_data=(x_test, y_test), epochs=100, verbose=2, batch_size=1024)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 100)          100000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 400, 64)           6464      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 57)                3705      
Total params: 110,169
Trainable params: 110,169
Non-trainable params: 0
_________________________________________________________________


C:\Users\iam\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 150000 samples, validate on 50000 samples
Epoch 1/100
 - 11s - loss: 4.0070 - categorical_accuracy: 0.0627 - val_loss: 3.9599 - val_categorical_accuracy: 0.1018
Epoch 2/100
 - 7s - loss: 3.8731 - categorical_accuracy: 0.1006 - val_loss: 3.7630 - val_categorical_accuracy: 0.1018
Epoch 3/100
 - 7s - loss: 3.6502 - categorical_accuracy: 0.1006 - val_loss: 3.5547 - val_categorical_accuracy: 0.1018
Epoch 4/100
 - 7s - loss: 3.5087 - categorical_accuracy: 0.1006 - val_loss: 3.4773 - val_categorical_accuracy: 0.1018
Epoch 5/100
 - 7s - loss: 3.4646 - categorical_accuracy: 0.1006 - val_loss: 3.4525 - val_categorical_accuracy: 0.1018
Epoch 6/100
 - 7s - loss: 3.4482 - categorical_accuracy: 0.1006 - val_loss: 3.4407 - val_categorical_accuracy: 0.1018
Epoch 7/100
 - 7s - loss: 3.4396 - categorical_accuracy: 0.1006 - val_loss: 3.4337 - val_categorical_accuracy: 0.1018
Epoch 8/100
 - 7s - loss: 3.4344 - categorical_accuracy: 0.1006 - val_loss: 3.4291 - val_categorical_accuracy: 0.1018
Epoc

KeyboardInterrupt: 

In [14]:
score = model.evaluate(x_test, y_test, verbose=0)
print(score)

[1.6326895734024047, 0.54422]


In [28]:
y_pred = np.argmax(model.predict(x_test), axis=-1)

In [29]:
arena_accuracy_score(y_pred, np.argmax(y_test, axis=-1))

array([0.5736])